# Import Packages

In [ ]:
import os, torch
import numpy as np
import pandas as pd
from PIL import Image
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Setup

In [ ]:
csv_file = "train.csv"
data_list = pd.read_csv(csv_file)
data_root = "train"
file = "{}.jpg"
data_list

# 0. Pipeline Illustration
關於訓練模型，我們有幾個步驟要做：
1. 讀資料、建構 Dataset 及 DataLoader
2. 定義 model, optimizer, loss
3. training and validation

# 1. Load Data and Construct Dataset, DataLoader

### 1.1. Load JPG, JPEG, PNG
因為我們的影像是 JPEG 檔，所以我們來學如何讀這系列的檔案

##### Note.
這系列の檔案類型都是色彩強度在 [0, 255] 的影像

In [ ]:
example_image = np.random.choice(data_list.StudyInstanceUID)
path = os.path.join(data_root, file.format(example_image))
path

In [ ]:
image = Image.open(path)
image

In [ ]:
array = np.array(image)
print(f"shape = {array.shape}")
np.unique(array)

### 1.2. Pytorch Dataset and DataLoader
Pytorch Dataset 是一種 iterable（熟悉吧？）
1. 它必須繼承 Pytorch 的 Dataset class
2. 是一個 iterable class，會一個一個吐出你的 data

至於 data 要用什麼形式包裝他就不限制了。
不過我想給大家一個管理 data 的建議：一個 data 用一個 dictionary 包裝。

In [ ]:
example_index = 0
data = {
    "patient_id": data_list.iloc[example_index, -1],
    "image": data_list.iloc[example_index, 0],
    "label": np.array(data_list.iloc[example_index, 1:-1], dtype="float32")
}
data

### 1.3. Your Trun!
請你動手寫一個名為 RANZCR 的 Pytorch Dataset。
1. 繼承 torch.utils.data.Dataset (Hint: 在開頭的 import 環節，我已經幫你把它 import 成 Dataset 了，繼承 Dataset 並 initialize 就好）
2. initialize 兩個字串 data_root 和 csv_file（建議按照順序）
  * 把 csv_file 讀成 pandas DataFrame 作為 attribute
  * 直接把 data_root 存成字串，用來讀串接影像名稱
3. 用你的方式寫一個 iterable：
  * 放到迴圈時，每一次迭代吐一組資料（即一個 dictionary 如上）
  * 可以取長度
  * 可以多次放到迴圈裡面使用

In [ ]:
# your code here

In [ ]:
check_answer = False

if check_answer:
    dataset = RANZCR(data_root, csv_file)

    for data in dataset:
        print(data)
        break

### 1.4. Data Transform
你肯定注意到了，我們讀進來的資料只有檔名，不可能直接拿來 train。
這時我們需要一連串的 transforms 來讓資料從檔名開始經歷他的奇幻旅程，這中間你愛加多少 data augmentation 都隨你開心。

In [ ]:
# 帶大家寫 Tranform 以及把 Dataset 修成有 transform 的版本

### 1.5. DataLoader
DataLoader 是 Pytorch 一個很方便的物件，它直接幫你把 Dataset 變成可以 batch-wise 讀取的迭代器，同時實現平行化讀取。

In [ ]:
loader = DataLoader(RANZCR, batch_size=32, shuffle=True, num_workers=0)

# 2. Setup Hyperparameters

### 2.1. Model Construction
Pytorhc model 是一種 torch.nn.Module class，並且有定義 __init__, forward
* __init__ 用來存 parameters
* forward 吃一個 input x，你必須指明 x 會經過哪些運算，最後 output 出去

##### Note.
但因為我幫你把 nn import 好了，所以你只需要寫 nn.Module 就可以使用它了

In [ ]:
class ExampleModel(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ExampleModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 16, kernel_size=3, stride=4, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=4, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=4, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.flat = nn.Flatten()
        self.linear = nn.Linear(256, out_channels)
        self.bn_out = nn.BatchNorm1d(out_channels)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.flat(x)
        x = F.sigmoid(self.bn_out(self.linear(x)))

        return x

In [ ]:
model = ExampleModel(1, 11)
model

In [ ]:
x = torch.rand(32, 1, 128, 128)
y = model(x)
y.size()

### 2.2. Optimizers and Losses

In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())

# 3. Training and Validation

### 3.1. Training Process
當我們的 data loader, model, criterion, optimizer 都設置好以後，接下來的 training process 就可以用底下幾個步驟概括了

0. 清空 optimizer 的 gradient 及將 model 改為訓練模式
  * optimizer.zero_grad()
  * model.train()
1. 從 data loader 取得一組資料
  * 資料 input
  * 標註 target
2. 計算 model 的預測
  * output = model(input)
3. 計算 loss
  * loss = criterion(output, target)
4. 計算 gradient
  * loss.backward()
5. 更新參數
  * optimizer.step()

### 3.2. Your Turn!
這邊留給你們寫應該不過分吧XD

In [ ]:
# your code here

### 3.3. Validation Process
同上設置，我們有底下步驟
0. 將 model 改為計算模式並且用 torch.no_grad() 包住整段 code
  * model.eval()
  * with torch.no_grad():
        ...
1. 從 data loader 取得一組資料
  * 資料 input
  * 標註 target
2. 計算 model 的預測
  * output = model(input)
3. 計算 loss 或你要的指標
  * loss = criterion(output, target)
  * metric = ...
4. 看你要 print 出來還是存到哪裡去都行1. 從 data loader 取得一組資料
  * 資料 input
  * 標註 target

### 3.4. Your Turn!
就算你覺得過分我也不會理你 <3

In [ ]:
# your code here